In [0]:
CREATE OR REFRESH STREAMING TABLE bronzee_results
COMMENT 'Raw customers data ingested from the source system operational data'
TBLPROPERTIES ('quality' = 'bronze')
AS 
SELECT *,
       _metadata.file_path AS input_file_path,
       CURRENT_TIMESTAMP AS ingestion_timestamp
FROM cloud_files(
  '/Volumes/ishwariformulabox/schemalanding/volume2/results.csv',
  'csv',
  map('cloudFiles.inferColumnTypes', 'true')
);

In [0]:
CREATE OR REFRESH STREAMING TABLE silver_race_results (
  CONSTRAINT valid_result_id EXPECT (resultId IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_race_id EXPECT (raceId IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_driver_id EXPECT (driverId IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_constructor_id EXPECT (constructorId IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_number EXPECT (number IS NOT NULL AND number > 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_grid EXPECT (grid IS NOT NULL AND grid > 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_position EXPECT (position IS NOT NULL AND position > 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_position_text EXPECT (positionText IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_position_order EXPECT (positionOrder IS NOT NULL AND positionOrder > 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_points EXPECT (points IS NOT NULL AND points >= 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_laps EXPECT (laps IS NOT NULL AND laps > 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_time EXPECT (time IS NOT NULL AND time REGEXP '^\\d{1,3}:\\d{2}\\.\\d$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_milliseconds EXPECT (milliseconds IS NOT NULL AND milliseconds >= 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_fastest_lap EXPECT (fastestLap IS NOT NULL) ON VIOLATION DROP ROW,
  CONSTRAINT valid_rank EXPECT (rank IS NOT NULL AND rank > 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_fastest_lap_time EXPECT (fastestLapTime IS NOT NULL AND fastestLapTime REGEXP '^\\d{2}:\\d{2}\\.\\d$') ON VIOLATION DROP ROW,
  CONSTRAINT valid_fastest_lap_speed EXPECT (fastestLapSpeed IS NOT NULL AND fastestLapSpeed >= 0) ON VIOLATION DROP ROW,
  CONSTRAINT valid_status_id EXPECT (statusId IS NOT NULL AND statusId > 0) ON VIOLATION DROP ROW
)
COMMENT 'Cleaned race results data'
TBLPROPERTIES ('quality' = 'silver')
AS
SELECT resultId,
       raceId,
       driverId,
       constructorId,
       number,
       grid,
       position,
       positionText,
       positionOrder,
       points,
       laps,
       time,
       milliseconds,
       fastestLap,
       rank,
       fastestLapTime,
       fastestLapSpeed,
       statusId
FROM STREAM(bronzee_results);
